# Generador de archivos para experimentar

In [83]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os

PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates-experimentos')),
    trim_blocks=False)

def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)
templateMA  = 'template-opinion-dueling-loops-conexion.ma'
templateEV  = 'template-events.ev'
templateVAL = 'template-valfile.val'
templateMACROS = 'template-macros.inc'

In [84]:
# Seteo nombre del directorio en el cual voy a guardar todos los experimentos
experiment_name = 'batch2'

# Seteo las dimensiones del cell-devs
N = 10
M = 10

# Seteo delay del cell-devs para que guarde una relacion que se condiga con la evolucion del modelo
# de arriba
delayValues = [100]

# 1 = envio shocks negativo / negativo / neutro / positivo, segun  limites 80 / 60 / 40 / 20 para Neutralists
# 2 = envio shocks neutro / positivo / neutro / negativo, segun limites 80 / 60 / 40 / 20 para Neutralists
shockCriteriaValues = [1, 2]

# InitialConditions: [%negative, %neutral, %positive]
initialConditionsValues = [
    [0.1, 0.1, 0.8],
    [0.1, 0.8, 0.1],
    
    [0.2, 0.4, 0.4],
    
    [0.4, 0.4, 0.2],
    [0.4, 0.2, 0.4],
    
    [0.8, 0.1, 0.1]
]

In [91]:
import numpy as np
from random import shuffle
def generateInitialValues(initialConditionValues, N, M):
    pnegative, pneutral, ppositive = initialConditionValues[0], initialConditionValues[1], initialConditionValues[2]
    negative, neutral = int(N*M*pnegative), int(N*M*pneutral)
    positive = N*M - neutral - negative
    vpositive = [np.random.uniform(1,3) for i in range(positive)]
    vneutral  = [np.random.uniform(-1,1) for i in range(neutral)]
    vnegative = [np.random.uniform(-3,-1) for i in range(negative)]
    v = vpositive + vneutral + vnegative
    shuffle(v)
    res = []
    for i in range(10):
        for j in range(10):
            res.append('(' + str(i) + ',' + str(j) + ',0)=' + str(v[M*i+j]))
    return res

In [92]:
#generateInitialValues([0.1,0.1,0.8], 10, 10)

In [93]:
nExperiment = 1
for delayValue in delayValues:
    for shockCriteriaValue in shockCriteriaValues:
        for initialConditionValues in initialConditionsValues:
            dir_name = experiment_name + '/' + str(delayValue) + '_' + str(shockCriteriaValue) + '_' + \
            str(initialConditionValues[0]) + '_' + str(initialConditionValues[1]) + \
            '_' + str(initialConditionValues[2])
            
            context = {
                'delayValue' : delayValue,
                'shockCriteriaValue' : shockCriteriaValue,
                'values' : []
            }

            
            # Generate directory if not exists
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
            
            with open(dir_name + '/opinion-combined-model.ma', 'w') as f:
                maFile = render_template(templateMA, context)
                f.write(maFile)
            with open(dir_name + '/events.ev', 'w') as f:
                evFile = render_template(templateEV, context)
                f.write(evFile)
            with open(dir_name + '/macros.inc', 'w') as f:
                macrosFile = render_template(templateMACROS, context)
                f.write(macrosFile)
                
            for experiment in range(0,30): 
                
                # Generate context
                context['values'] = generateInitialValues(initialConditionValues, N, M)

                # Generate file
                with open('{0}/valfile_exp_{1:02d}.val'.format(dir_name, experiment), 'w') as f:
                    valFile = render_template(templateVAL, context)
                    f.write(valFile)
                nExperiment += 1